NoteBook for creating Model
------------------------------------
12.07.2025

In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os
import tensorflow as tf

2025-11-23 00:18:48.361233: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 00:18:48.562514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763875128.615628    4092 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763875128.630699    4092 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763875128.826813    4092 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
#Downlaods dataset and sets it to path
path = kagglehub.dataset_download("sanikamal/rock-paper-scissors-dataset")
print(path)
data_root = path + "/rock-paper-scissors/Rock-Paper-Scissors"
#makes a directory for each
train_dir = data_root + "/train"
val_dir   = data_root + "/validation"
test_dir  = data_root + "/test"

/root/.cache/kagglehub/datasets/sanikamal/rock-paper-scissors-dataset/versions/1


Now turn the images into TensorFlow training data-set

In [3]:
img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)


Found 2520 files belonging to 3 classes.
Using 2016 files for training.


I0000 00:00:1763875136.522112    4092 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 2520 files belonging to 3 classes.
Using 504 files for validation.
Found 372 files belonging to 3 classes.


Optimize and Normalize data before training the Model


In [4]:
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(ds):
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
    return ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

train_ds = preprocess(train_ds)
val_ds = preprocess(val_ds)
test_ds = preprocess(test_ds)

Creating the CNN model with 3 layers

In [11]:
model = tf.keras.Sequential([
    # First Layer 
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    # Second Layer 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    #Third Layer 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    #fourth Layer
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),


    #Regularization and dense part
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

/root/miniconda3/envs/FINALCIS/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compile the Model

In [13]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


Train based off preprocessed data

In [20]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25
)

Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 99ms/step - accuracy: 1.0000 - loss: 3.0748e-08 - val_accuracy: 1.0000 - val_loss: 2.8383e-09
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 1.0000 - loss: 3.3232e-08 - val_accuracy: 1.0000 - val_loss: 2.3653e-09
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 1.0000 - loss: 2.2825e-08 - val_accuracy: 1.0000 - val_loss: 2.1287e-09
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - accuracy: 1.0000 - loss: 1.6439e-08 - val_accuracy: 1.0000 - val_loss: 2.1287e-09
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 1.0000 - loss: 1.3127e-08 - val_accuracy: 1.0000 - val_loss: 2.1287e-09
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 1.0000 - loss: 2.4480e-08 - val_accuracy: 1.0000 - val_loss: 1.6557e-09
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 1.0000 - loss: 1.3541e-08 - val_accuracy: 1.0000 - val_loss: 1.6557e-09
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - a

In [22]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.2f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7823 - loss: 3.5868
Test accuracy: 0.78
